# Implementación de máxima verosimilitud para la regresión logística

In [1]:
import pandas as pd
import numpy as np

## Definir la función de "entorno" $\mathcal{L}(b)$

La verosimilitud será en cierta manera igual a la probabilidad de encontrar la muestra que se ha extraído en función de los parámetros.
$$L(\beta) = \sum_{i = 1}^{n}p_i^{y_i}(1-p_i)^{1-y_i}$$

In [2]:
def likelihood(y, pi):
    n = len(pi)
    m = len(y)
    
    if n != m:
        print("Error: Observation and probability vector have different lengths.")
        return 0
    else:
        like = 0
        term = list(np.repeat(0,n))
        
        for i in range(n):
            term[i] = np.where(y[i] == 1, pi[i], 1-pi[i])
            like = like + term[i]
        return like

## Calcular las probabilidades para cada observación

Recordar que las probabilidades para el modelo logístico se hallaban de la siguiente manera:

$$ p_i := P(y_i = 1\vert x = x_0) = \frac{1}{1+e^{-\beta' x_0}} $$

In [3]:
def logitprobs(X, beta):
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    
    pi = list(np.repeat(0, n_rows))
    expon = list(np.repeat(0, n_rows))
    
    for i in range(n_rows):
        for j in range(n_cols):
            ex = X[i,j] * beta[j,0]
            expon[i] = ex + expon[i]
            
        with np.errstate(divide = "ignore", invalid = "ignore"):
            pi[i] = 1/(1+np.exp(-expon[i]))
    
    return pi

## Calcular la matriz diagonal $W$

$$ W = \text{diag}\left( p_1(1-p_1), \ldots, p_n(1-p_n)\right)$$

In [4]:
def findW(pi):
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        W[i,i] = float(pi[i] * (1 - pi[i]))
    return W

### Obtener la solución de la función logística

Obtenido del método de Newton-Raphson:

$$ \hat\beta_{n} = \hat\beta_{n-1} - \frac{f(\hat\beta_{n-1})}{f'(\hat\beta_{n-1})}$$

In [5]:
def logistic(X, Y, limit):
    n_row = np.shape(X)[0]
    intcp = np.ones(n_row).reshape(n_row,1)
    X_new = np.append(X, intcp, axis = 1)
    n_col = np.shape(X_new)[1]
    beta = np.zeros(n_col).reshape(n_col,1)
    root_dif = np.array(range(1, n_col + 1)).reshape(n_col,1)
    iter_i = 10000
    i = 1
    while (iter_i > limit):
        if i == 1:
            print("Iteration ", i,".", sep = "")
        else:
            print("Iteration ", i,".", sep = "")
        pi = logitprobs(X_new, beta)
        W = findW(pi)
        num = np.transpose(np.matrix(X_new)) * np.matrix(Y - np.transpose(pi)).transpose()
        den = np.matrix(np.transpose(X_new) * np.matrix(W) * np.matrix(X_new))
        root_dif = np.array(np.linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: \n",beta, sep = "")
        iter_i = np.sum(root_dif * root_dif)
        print("Magnitude of error: {:.4f}".format(iter_i),"\n",sep = "")
        ll = likelihood(Y, pi)
        i += 1
    return beta

## Comprobación experimental

In [6]:
X = np.array(range(10)).reshape(10,1)
Y = [0,0,0,0,1,0,1,0,1,1]

In [7]:
logistic(X, Y, 0.0000001)

Iteration 1.
Beta: 
[[ 0.43636364]
 [-2.36363636]]
Magnitude of error: 5.7772

Iteration 2.
Beta: 
[[ 0.60426056]
 [-3.34641372]]
Magnitude of error: 0.9940

Iteration 3.
Beta: 
[[ 0.65761412]
 [-3.66759924]]
Magnitude of error: 0.1060

Iteration 4.
Beta: 
[[ 0.66217766]
 [-3.6953843 ]]
Magnitude of error: 0.0008

Iteration 5.
Beta: 
[[ 0.66220827]
 [-3.69557172]]
Magnitude of error: 0.0000



array([[ 0.66220827],
       [-3.69557172]])

## Con el paquete statsmodel

In [9]:
import statsmodels.api as sm

In [15]:
X_new = np.c_[[1]*10,X]

In [17]:
logit_model = sm.Logit(Y,X_new)

In [18]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [19]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Thu, 13 Jan 2022   Pseudo R-squ.:                  0.3596
Time:                        00:28:06   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                   0.02781
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.6956      2.289     -1.615      0.106      -8.182       0.791
x1             0.6622      0.400      1.655      0.098      -0.122       1.446
==============================================================================
"""